In [1]:
import numpy as np
import pandas as pd
import matplotlib
import datetime
import time
from itertools import islice

In [2]:
def parse_timestamp(el):
    b = datetime.datetime.strptime(el, fmt)
    return b

In [3]:
fmt = '%d.%m.%Y'

In [4]:
# Importing data on countermeasures
url = "covid19_unified_set.csv"
complete_df = pd.read_csv(url, sep=';', low_memory=False)

In [5]:
pd.set_option('display.max_columns()', None)

In [6]:
complete_df

,date,cultural_venues,gastronomy,essential_retail,non-essential_retail,major_events,events_and_gatherings,public_gatherings,traceability,tourism,traveling,school_facilities,university_facilities,sport_facilities,medical_facilities,mouth_and_nose_covering,curfew,new_infections
0,2020-01-01,open,open,open,open,allowed,allowed,allowed,no,allowed,allowed,open,open,open,open,not_mandatory,no,0
1,2020-01-02,open,open,open,open,allowed,allowed,allowed,no,allowed,allowed,open,open,open,open,not_mandatory,no,0
2,2020-01-03,open,open,open,open,allowed,allowed,allowed,no,allowed,allowed,open,open,open,open,not_mandatory,no,0
3,2020-01-04,open,open,open,open,allowed,allowed,allowed,no,allowed,allowed,open,open,open,open,not_mandatory,no,0
4,2020-01-05,open,open,open,open,allowed,allowed,allowed,no,allowed,allowed,open,open,open,open,not_mandatory,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021-07-27,openw/meas,openw/meas,openw/tst_vacc,openw/lim_ppl,sport_1000,allowed_1000max,allowed_100ppl_tst_imm,NaN,allowed,ra_testM_vac_10dqua,open,open,openw/limit,limited_visitsw/mask_test,mandatory_medical,no,82
574,2021-07-28,openw/meas,openw/meas,openw/tst_vacc,openw/lim_ppl,sport_1000,allowed_1000max,allowed_100ppl_tst_imm,NaN,allowed,ra_testM_vac_10dqua,open,open,openw/limit,limited_visitsw/mask_test,mandatory_medical,no,65
575,2021-07-29,openw/meas,openw/meas,openw/tst_vacc,openw/lim_ppl,sport_1000,allowed_1000max,allowed_100ppl_tst_imm,NaN,allowed,ra_testM_vac_10dqua,open,open,openw/limit,limited_visitsw/mask_test,mandatory_medical,no,74
576,2021-07-30,openw/meas,openw/meas,openw/tst_vacc,openw/lim_ppl,sport_1000,allowed_1000max,allowed_100ppl_tst_imm,NaN,allowed,ra_testM_vac_10dqua,open,open,openw/limit,limited_visitsw/mask_test,mandatory_medical,no,50


In [7]:
complete_df.columns

Index(['date', 'cultural_venues', 'gastronomy', 'essential_retail',
       'non-essential_retail', 'major_events', 'events_and_gatherings',
       'public_gatherings', 'traceability', 'tourism', 'traveling',
       'school_facilities', 'university_facilities', 'sport_facilities',
       'medical_facilities', 'mouth_and_nose_covering', 'curfew',
       'new_infections'],
      dtype='object')

## Transformation of features

### Ordinal encoding

In [8]:
#1
complete_df['cultural_venues'].value_counts(dropna=False)

closed               175
openw/meas           142
openw/lim_100         93
openw/appointment     86
open                  75
closedw/exmz           7
Name: cultural_venues, dtype: int64

In [9]:
#replacing unnecesary and rare values
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('closedw/exmz','closed')

In [10]:
#converting to integers - 5 point scale
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('closed','5')
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('openw/appointment','4')
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('openw/lim_100','3')
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('openw/meas','2')
complete_df['cultural_venues'] = complete_df['cultural_venues'].str.replace('open','1')

In [11]:
#2
complete_df['gastronomy'].value_counts(dropna=False)

closedw/dp         263
openw/meas         228
open                75
openw/tst_vacc       7
outside_imun/dp      5
Name: gastronomy, dtype: int64

In [12]:
#replacing unnecesary and rare values
complete_df['gastronomy'] = complete_df['gastronomy'].str.replace('outside_imun/dp','openw/tst_vacc')

## Nema smisla da gastronomy bude open bez dokaza a essential retail ne, ispravi to u podacima

In [13]:
complete_df['gastronomy'] = complete_df['gastronomy'].str.replace('closedw/dp','4')
complete_df['gastronomy'] = complete_df['gastronomy'].str.replace('openw/tst_vacc','3')
complete_df['gastronomy'] = complete_df['gastronomy'].str.replace('openw/meas','2')
complete_df['gastronomy'] = complete_df['gastronomy'].str.replace('open','1')

In [14]:
#3
complete_df['essential_retail'].value_counts(dropna=False)

openw/meas            355
openw/lim_1p_10sqm     88
open                   75
openw/tst_vacc         60
Name: essential_retail, dtype: int64

In [15]:
complete_df['essential_retail'] = complete_df['essential_retail'].str.replace('openw/tst_vacc','4')
complete_df['essential_retail'] = complete_df['essential_retail'].str.replace('openw/lim_1p_10sqm','3')
complete_df['essential_retail'] = complete_df['essential_retail'].str.replace('openw/meas','2')
complete_df['essential_retail'] = complete_df['essential_retail'].str.replace('open','1')

In [16]:
#4
complete_df['non-essential_retail'].value_counts(dropna=False)

openw/meas                  175
open                         75
closed                       74
openw/lim_ppl                65
closedw/delivery             56
openw/lim_1p_10sqm           44
openw/app_negtst             35
openw/appointment            28
openw/lim_1p_10sqm_esent     26
Name: non-essential_retail, dtype: int64

In [17]:
#replacing unnecesary and rare values
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/lim_1p_10sqm','openw/lim_ppl')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/lim_1p_10sqm_esent','openw/lim_ppl')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/app_negtst','openw/appointment')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/lim_ppl_esent','openw/lim_ppl')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('closedw/delivery','closed')

In [18]:
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('closed','5')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/appointment','4')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/lim_ppl','3')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('openw/meas','2')
complete_df['non-essential_retail'] = complete_df['non-essential_retail'].str.replace('open','1')

In [19]:
#5
complete_df['major_events'].value_counts(dropna=False)

prohibited    443
allowed        75
sport_1000     60
Name: major_events, dtype: int64

In [20]:
complete_df['major_events'] = complete_df['major_events'].str.replace('prohibited','3')
complete_df['major_events'] = complete_df['major_events'].str.replace('sport_1000','2')
complete_df['major_events'] = complete_df['major_events'].str.replace('allowed','1')

In [21]:
#6
complete_df['events_and_gatherings'].value_counts(dropna=False)

prohibitedw/ex_trials      144
pw/ex_concert_100           93
allowedw/meas               82
allowed                     75
prohibitedw/ex_outdoors     63
prohibited                  56
allowed_1000max             53
allowed_500max               7
allowed_250max               5
Name: events_and_gatherings, dtype: int64

In [22]:
#replacing unnecesary and rare values
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowed_1000max','allowedw/lim_ppl')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowed_500max','allowedw/lim_ppl')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowed_250max','allowedw/lim_ppl')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('prohibitedw/ex_trials','prohibited')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('prohibitedw/ex_outdoors','prohibitedw/ex')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('pw/ex_concert_100','prohibitedw/ex')

In [23]:
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('prohibitedw/ex','4')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('prohibited','5')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowedw/lim_ppl','3')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowedw/meas','2')
complete_df['events_and_gatherings'] = complete_df['events_and_gatherings'].str.replace('allowed','1')

In [24]:
#7
complete_df['public_gatherings'].value_counts(dropna=False)

allowed_2h                 175
allowed_2h_1p               89
allowed_2h_5ppl             80
allowed                     75
prohibited_over_2           56
allowed_100ppl_tst_imm      52
allowed_10p                 29
allowed_10ppl                9
allowed_10ppl_3h_imm         7
allowed_2h_imun              5
allowed_100pplw/tst_imm      1
Name: public_gatherings, dtype: int64

In [25]:
#replacing unnecesary and rare values
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_2h_imun','allowed_2h_imm')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_2h_5ppl','allowed_2h')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_2h_1p','allowed_2h')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_10ppl_3h_imm','allowed_2h_imm')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_10ppl','allowed_2h')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_10p','allowed_2h')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_100pplw/tst_imm','allowed_100p')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_100ppl_tst_imm','allowed_100p')

In [26]:
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('prohibited_over_2','5')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_2h_imm','3')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_2h','4')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed_100p','2')
complete_df['public_gatherings'] = complete_df['public_gatherings'].str.replace('allowed','1')

In [27]:
#9
complete_df['tourism'].value_counts(dropna=False)

prohibited                 269
allowed                    128
allowed_only4               94
allowedw/ex_ra_med_cert     62
limited                     13
allowedw/test                7
limitedw/test                5
Name: tourism, dtype: int64

In [28]:
#replacing unnecesary and rare values
complete_df['tourism'] = complete_df['tourism'].str.replace('allowedw/ex_ra_med_cert','allowedw/test')
complete_df['tourism'] = complete_df['tourism'].str.replace('limitedw/test','allowedw/test')
complete_df['tourism'] = complete_df['tourism'].str.replace('limited','prohibited')

In [29]:
complete_df['tourism'] = complete_df['tourism'].str.replace('prohibited','4')
complete_df['tourism'] = complete_df['tourism'].str.replace('allowed_only4','3')
complete_df['tourism'] = complete_df['tourism'].str.replace('allowedw/test','2')
complete_df['tourism'] = complete_df['tourism'].str.replace('allowed','1')

In [30]:
#10
complete_df['traveling'].value_counts(dropna=False)

ra_10dqua_reg_5testM           100
ra_48test_14dqpos               95
allowed                         75
ra_testM_vac_10dqua             74
ra_14d_quarantine               65
ra_10dqua_reg_5test             42
o4_14d_quarantine               36
ra_14d_limitations              25
ra_testM_10dqua                 22
ra_14d_nomedfac                 17
above canceled                  15
ra_10dqua_reg_5test_mUKSA        7
ra_testM_vac_14dqua_noshort      3
ra_10dqua_postestM               2
Name: traveling, dtype: int64

In [31]:
#replacing unnecesary and rare values
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_10dqua_reg_5test_mUKSA','10q_r_5')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_10dqua_reg_5testM','10q_r_5')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_10dqua_reg_5test','10q_r_5')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_testM_vac_10dqua','10q_proof')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_10dqua_postestM','10q_proof')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_testM_10dqua','10q_proof')
complete_df['traveling'] = complete_df['traveling'].str.replace('above canceled','10q_proof')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_testM_vac_14dqua_noshort','14q')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_14d_quarantine','14q')
complete_df['traveling'] = complete_df['traveling'].str.replace('o4_14d_quarantine','14q')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_48test_14dqpos','14q')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_14d_limitations','14q_light')
complete_df['traveling'] = complete_df['traveling'].str.replace('ra_14d_nomedfac','14q_light')

In [32]:
complete_df['traveling'] = complete_df['traveling'].str.replace('14q_light','2')
complete_df['traveling'] = complete_df['traveling'].str.replace('14q','5')
complete_df['traveling'] = complete_df['traveling'].str.replace('10q_proof','4')
complete_df['traveling'] = complete_df['traveling'].str.replace('10q_r_5','3')
complete_df['traveling'] = complete_df['traveling'].str.replace('allowed','1')

In [33]:
#11
complete_df['school_facilities'].value_counts(dropna=False)

openw/meas       335
open             128
closed            64
openw/tst_vac     39
openw/test        12
Name: school_facilities, dtype: int64

In [34]:
#replacing unnecesary and rare values
complete_df['school_facilities'] = complete_df['school_facilities'].str.replace('openw/test','openw/tst_vac')

In [35]:
complete_df['school_facilities'] = complete_df['school_facilities'].str.replace('closed','4')
complete_df['school_facilities'] = complete_df['school_facilities'].str.replace('openw/tst_vac','3')
complete_df['school_facilities'] = complete_df['school_facilities'].str.replace('openw/meas','2')
complete_df['school_facilities'] = complete_df['school_facilities'].str.replace('open','1')

In [36]:
#12
complete_df['university_facilities'].value_counts(dropna=False)

openw/meas       252
open             128
open_limited     102
closedw/exlib     49
openw/tst_vac     39
closed             8
Name: university_facilities, dtype: int64

In [37]:
#replacing unnecesary and rare values
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('closedw/exlib','closed')

In [38]:
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('closed','5')
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('open_limited','4')
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('openw/tst_vac','3')
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('openw/meas','2')
complete_df['university_facilities'] = complete_df['university_facilities'].str.replace('open','1')

In [39]:
#13
complete_df['sport_facilities'].value_counts(dropna=False)

closed          194
openw/meas      175
openw/limit     130
open             75
closedw/expo      4
Name: sport_facilities, dtype: int64

In [40]:
#replacing unnecesary and rare values
complete_df['sport_facilities'] = complete_df['sport_facilities'].str.replace('closedw/expo','closed')

In [41]:
complete_df['sport_facilities'] = complete_df['sport_facilities'].str.replace('closed','4')
complete_df['sport_facilities'] = complete_df['sport_facilities'].str.replace('openw/limit','3')
complete_df['sport_facilities'] = complete_df['sport_facilities'].str.replace('openw/meas','2')
complete_df['sport_facilities'] = complete_df['sport_facilities'].str.replace('open','1')

In [42]:
#14
complete_df['medical_facilities'].value_counts(dropna=False)

allowed_visitsw/meas         219
limited_visitsw/FFP2_test    125
open                          75
limited_visits                56
limited_visitsw/FFP2          40
limited_visitsw/mask_test     37
allowed_visitsw/FFP2          26
Name: medical_facilities, dtype: int64

In [43]:
#replacing unnecesary and rare values
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('limited_visitsw/FFP2_test','limited_visitsw/mask_test')
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('allowed_visitsw/FFP2','allowed_visitsw/meas')

In [44]:
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('limited_visitsw/mask_test','5')
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('limited_visitsw/FFP2','4')
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('limited_visits','3')
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('allowed_visitsw/meas','2')
complete_df['medical_facilities'] = complete_df['medical_facilities'].str.replace('open','1')

In [45]:
#15
complete_df['mouth_and_nose_covering'].value_counts(dropna=False)

not_mandatory                224
mandatory_textile            112
mandatory_medical            105
mandatory_textile_no_dist     82
mandatory_mixed               55
Name: mouth_and_nose_covering, dtype: int64

In [46]:
#replacing unnecesary and rare values
complete_df['mouth_and_nose_covering'] = complete_df['mouth_and_nose_covering'].str.replace('mandatory_textile_no_dist','mandatory_textile')

In [47]:
complete_df['mouth_and_nose_covering'] = complete_df['mouth_and_nose_covering'].str.replace('mandatory_medical','4')
complete_df['mouth_and_nose_covering'] = complete_df['mouth_and_nose_covering'].str.replace('mandatory_mixed','3')
complete_df['mouth_and_nose_covering'] = complete_df['mouth_and_nose_covering'].str.replace('mandatory_textile','2')
complete_df['mouth_and_nose_covering'] = complete_df['mouth_and_nose_covering'].str.replace('not_mandatory','1')

In [48]:
complete_df

,date,cultural_venues,gastronomy,essential_retail,non-essential_retail,major_events,events_and_gatherings,public_gatherings,traceability,tourism,traveling,school_facilities,university_facilities,sport_facilities,medical_facilities,mouth_and_nose_covering,curfew,new_infections
0,2020-01-01,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
1,2020-01-02,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
2,2020-01-03,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
3,2020-01-04,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
4,2020-01-05,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021-07-27,2,2,4,3,2,3,2,NaN,1,4,1,1,3,5,4,no,82
574,2021-07-28,2,2,4,3,2,3,2,NaN,1,4,1,1,3,5,4,no,65
575,2021-07-29,2,2,4,3,2,3,2,NaN,1,4,1,1,3,5,4,no,74
576,2021-07-30,2,2,4,3,2,3,2,NaN,1,4,1,1,3,5,4,no,50


### Dummy encoding

In [49]:
#8
complete_df['traceability'].value_counts(dropna=False)

yes    369
no     150
NaN     59
Name: traceability, dtype: int64

In [50]:
#replacing unnecesary and rare values
complete_df['traceability'] = complete_df['traceability'].fillna('yes')

In [51]:
#16
complete_df['curfew'].value_counts(dropna=False)

no                538
10pm_5am_12act     40
Name: curfew, dtype: int64

In [52]:
complete_df['curfew'] = complete_df['curfew'].str.replace('10pm_5am_12act','yes')

In [53]:
trccols = pd.get_dummies(complete_df['traceability'], prefix='traceability')

In [54]:
crfcols = pd.get_dummies(complete_df['curfew'], prefix='curfew')

In [55]:
crfcols

,curfew_no,curfew_yes
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
573,1,0
574,1,0
575,1,0
576,1,0


In [56]:
complete_df

,date,cultural_venues,gastronomy,essential_retail,non-essential_retail,major_events,events_and_gatherings,public_gatherings,traceability,tourism,traveling,school_facilities,university_facilities,sport_facilities,medical_facilities,mouth_and_nose_covering,curfew,new_infections
0,2020-01-01,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
1,2020-01-02,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
2,2020-01-03,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
3,2020-01-04,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
4,2020-01-05,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021-07-27,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,82
574,2021-07-28,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,65
575,2021-07-29,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,74
576,2021-07-30,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,50


In [57]:
final_df = pd.concat([complete_df, trccols], axis=1)
final_df = pd.concat([final_df, crfcols], axis=1)

In [58]:
final_df

,date,cultural_venues,gastronomy,essential_retail,non-essential_retail,major_events,events_and_gatherings,public_gatherings,traceability,tourism,traveling,school_facilities,university_facilities,sport_facilities,medical_facilities,mouth_and_nose_covering,curfew,new_infections,traceability_no,traceability_yes,curfew_no,curfew_yes
0,2020-01-01,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0,1,0,1,0
1,2020-01-02,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0,1,0,1,0
2,2020-01-03,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0,1,0,1,0
3,2020-01-04,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0,1,0,1,0
4,2020-01-05,1,1,1,1,1,1,1,no,1,1,1,1,1,1,1,no,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,2021-07-27,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,82,0,1,1,0
574,2021-07-28,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,65,0,1,1,0
575,2021-07-29,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,74,0,1,1,0
576,2021-07-30,2,2,4,3,2,3,2,yes,1,4,1,1,3,5,4,no,50,0,1,1,0


In [59]:
final_df.drop(['traceability', 'curfew'], axis=1, inplace=True)

In [60]:
final_df.to_csv('covid19_final_set.csv', sep=';', index=False)